## 1. tf.keras.Sequential 简介
Sequential 是 Keras 中用于快速搭建层叠式（线性堆叠）神经网络模型的容器。它允许你按顺序一层层地添加网络层，适合简单的前馈神经网络结构。

## 2. Sequential 的初始化参数及用法示例

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# 方法1：传入层列表（list）
model1 = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(100,)),
    layers.Dense(10, activation='softmax')
])

# 方法2：空模型，后续用 add() 添加层
model2 = tf.keras.Sequential()
model2.add(layers.Dense(64, activation='relu', input_shape=(100,)))
model2.add(layers.Dense(10, activation='softmax'))

# 方法3：传入命名参数构建空模型
model3 = tf.keras.Sequential(name='my_sequential_model')


## 3. 构造函数参数详细表
| 参数名      | 类型               | 默认值    | 说明                                                    |
| -------- | ---------------- | ------ | ----------------------------------------------------- |
| `layers` | list 或 tuple（可选） | `None` | 一组层对象（如 `Dense`, `Conv2D` 等），按顺序堆叠组成模型。如果传入，模型即含有这些层。 |
| `name`   | str（可选）          | 自动生成   | 模型名称，方便标识和保存。                                         |
## 4. 重要方法
| 方法                                       | 参数                                 | 说明                        |
| ---------------------------------------- | ---------------------------------- | ------------------------- |
| `add(layer)`                             | `layer`：`tf.keras.layers.Layer` 对象 | 向模型末尾追加一层。适合初始化时空模型，逐步构建。 |
| `compile(optimizer, loss, metrics, ...)` | 常用训练参数                             | 配置训练过程。                   |
| `fit(x, y, epochs, batch_size, ...)`     | 训练参数                               | 训练模型。                     |
| `evaluate(x, y, batch_size, ...)`        | 评估参数                               | 测试模型。                     |
| `predict(x, batch_size, ...)`            | 预测参数                               | 预测新数据。                    |
| `summary()`                              | 无                                  | 打印模型结构。                   |

## 5. 代码示例说明

In [ ]:
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

model.summary()

该模型是一个简单的全连接网络：
第一层：Dense(128)，输入 784 维，激活 ReLU。
第二层：Dropout(0.2)，训练时随机丢弃 20% 神经元，防止过拟合。
第三层：Dense(10)，输出 10 个类别概率，用 softmax 激活。

## 6. 关键参数详解表（部分常用层参数）
| 参数名           | 说明                         | 适用层举例              |
| ------------- | -------------------------- | ------------------ |
| `units`       | 神经元个数（输出维度）                | `Dense`            |
| `activation`  | 激活函数名称或函数                  | 几乎所有层              |
| `input_shape` | 输入数据形状（不含批次大小）             | 第一层必须指定，如 `(784,)` |
| `rate`        | Dropout 丢弃比例（0\~1）         | `Dropout`          |
| `filters`     | 卷积核数量                      | `Conv2D`           |
| `kernel_size` | 卷积核尺寸                      | `Conv2D`           |
| `padding`     | 边缘填充方式（`'valid'`或`'same'`） | `Conv2D`           |

## 总结
| 特点    | 说明                                        |
| ----- | ----------------------------------------- |
| 适合场景  | 简单线性堆叠模型，无分支或多输入输出。                       |
| 层添加方式 | 构造时传列表 或 先建空模型后用 `.add()` 逐层添加。           |
| 灵活性   | 不支持复杂拓扑结构（如跳跃连接、残差、多个输入输出），需要用子类化或函数式API。 |


# 使用 tf.keras.Sequential() 建模示例

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# 1. 加载并预处理数据
# 使用MNIST手写数字数据集
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 将图像数据归一化到0-1范围
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 将图像数据从(28, 28)重塑为(28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# 将标签转换为one-hot编码
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 2. 使用Sequential构建模型
model = tf.keras.Sequential()

# 添加卷积层
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

# 添加另一个卷积层
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# 添加Dropout层防止过拟合
model.add(layers.Dropout(0.25))

# 将多维数据展平为一维
model.add(layers.Flatten())

# 添加全连接层
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))

# 添加输出层，使用softmax激活函数进行多分类
model.add(layers.Dense(10, activation='softmax'))

# 3. 编译模型
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 4. 训练模型
history = model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=10,
    validation_split=0.1  # 使用10%的训练数据作为验证集
)

# 5. 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'\n测试准确率: {test_acc:.4f}')

# 6. 绘制训练过程
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='训练准确率')
plt.plot(history.history['val_accuracy'], label='验证准确率')
plt.title('模型准确率')
plt.xlabel('Epoch')
plt.ylabel('准确率')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='训练损失')
plt.plot(history.history['val_loss'], label='验证损失')
plt.title('模型损失')
plt.xlabel('Epoch')
plt.ylabel('损失')
plt.legend()

plt.tight_layout()
plt.show()

# 7. 使用模型进行预测
# 从测试集中随机选择一个样本
sample_idx = np.random.randint(0, len(x_test))
sample_image = x_test[sample_idx]
sample_label = np.argmax(y_test[sample_idx])

# 扩展维度以匹配模型输入要求
sample_image = np.expand_dims(sample_image, axis=0)

# 进行预测
prediction = model.predict(sample_image)
predicted_label = np.argmax(prediction)

print(f'\n实际标签: {sample_label}')
print(f'预测标签: {predicted_label}')
print(f'预测置信度: {np.max(prediction):.4f}')

# 显示测试图像
plt.imshow(np.squeeze(sample_image), cmap='gray')
plt.title(f'实际: {sample_label}, 预测: {predicted_label}')
plt.axis('off')
plt.show()